# Evaluation
This script calculates the performance of all approaches in an output folder. Following metrics are calculated:
- PR AUC
- Best Precision, Recall, F0.5
- NAB

In [28]:
import os
import json
from copy import deepcopy
import numpy as np
import pandas as pd
from sklearn.metrics import auc

## Paths

In [29]:
tuning_job_name = 'streaming-SMD-240307-1040'
dataset_category, collection_id, further = 'multivariate', 'SMD', f'/{tuning_job_name}/SMD'
output_folder_path = f'../out/{dataset_category}/{collection_id}{further}'
data_folder_base_path = f'../data/{dataset_category}/{collection_id}'
dataset_ids = [x for x in os.listdir(output_folder_path) if not x.startswith('.')]

## Evaluation metrics

In [30]:
def calculate_anomaly_sequences(target):
    anomaly_sequences = []
    anomaly_indices = np.unique(np.where(target == 1)[0])
    change_ind = np.where(np.diff(anomaly_indices) != 1)[0] + 1
    if len(change_ind) != 0:
        sequences = np.split(anomaly_indices, change_ind)
    else:
        sequences = [anomaly_indices]
    for sequence in sequences:
        if len(sequence) != 0:
            anomaly_sequences.append([np.min(sequence), np.max(sequence)])
    return anomaly_sequences

In [31]:
def calculate_anomaly_sequences_2d(targets):
    assert len(targets.shape) == 2   # first dimension is threshold dimension
    anomaly_sequences = [[] for i in range(len(targets))]
    sequences = []
    for i in range(len(targets)):
        anomaly_indices = np.where(targets[i] == 1)[0]
        change_ind = np.where(np.diff(anomaly_indices) != 1)[0] + 1
        if len(change_ind) != 0:
            sequences.extend(np.split(np.stack([i * np.ones_like(anomaly_indices), anomaly_indices], axis=0), change_ind, axis=1))
        else:
            sequences.append(np.stack([i * np.ones_like(anomaly_indices), anomaly_indices], axis=0))
    # print(f'There are {len(sequences)} sequences')
    for sequence in sequences:
        if sequence.size != 0:
            threshold_idx = int(sequence[0, 0])
            anomaly_sequences[threshold_idx].append([np.min(sequence[1, :]), np.max(sequence[1, :])])
    return anomaly_sequences

In [32]:
def overlap(start1, end1, start2, end2):
    """Does the range (start1, end1) overlap with (start2, end2)?"""
    return not (end1 < start2 or end2 < start1)

In [33]:
def pr_auc(anomaly_scores, labels, return_best_threshold_precision_recall=True):
    assert len(anomaly_scores.shape) == 1
    thresholds = np.arange(0, 1, 0.01)

    # 1) Calculate binary prediction for every threshold
    predictions_binary = np.stack([anomaly_scores > thresholds[i] for i in range(len(thresholds))], axis=0)

    # 2) Smoothing (optional)

    # 3) Calculate anomaly sequences
    pred_anomaly_sequences = calculate_anomaly_sequences_2d(predictions_binary)
    true_anomaly_sequences = calculate_anomaly_sequences(labels)

    # 4) Calculate TP, FN, FP for every threshold
    true_positives, false_negatives, false_positives = [], [], []
    precisions, recalls = [], []
    for i in range(len(thresholds)):
        TP, FN, FP = 0, 0, 0
        pred_anomaly_sequences_single_threshold_copy = pred_anomaly_sequences[i].copy()
        for true_sequence in true_anomaly_sequences:
            # Check whether there is overlap with any pred_sequence
            overlap_list = [(pred_sequence, overlap(true_sequence[0], true_sequence[1], pred_sequence[0], pred_sequence[1])) for pred_sequence in pred_anomaly_sequences_single_threshold_copy]
            if any([x[1] for x in overlap_list]):
                TP += 1
                for pred_sequence in [x[0] for x in overlap_list if x[1]]:
                    # print(f'{pred_sequence} overlapping with {true_sequence}')
                    pred_anomaly_sequences_single_threshold_copy.remove(pred_sequence)
            else:
                FN += 1
        FP = len(pred_anomaly_sequences_single_threshold_copy)
        true_positives.append(TP)
        false_negatives.append(FN)
        false_positives.append(FP)
        precisions.append(TP / max(1, TP + FP))
        recalls.append(TP / max(1, TP + FN))

    # 5) Calculate AUC
    precisions, recalls = np.array(precisions), np.array(recalls)
    # auc = np.sum(np.abs(recalls[1:] - recalls[:-1]) * precisions[1:])
    sort_ind = np.argsort(recalls)
    auc_score = auc(recalls[sort_ind], precisions[sort_ind])

    # 6) Find best threshold
    if return_best_threshold_precision_recall:
        idx = np.argmax(np.square(precisions) + np.square(recalls))
        return auc_score, thresholds[idx], precisions[idx], recalls[idx]
    else:
        return auc_score

In [34]:
def nab_scoring_function(y):
    return 2 * (1 / (1 + np.exp(5*y))) - 1

In [35]:
def nab_scoring(anomaly_scores, labels, return_threshold=True):
    assert len(anomaly_scores.shape) == 1
    thresholds = np.arange(0, 1, 0.01)

    # 1) Calculate binary prediction for every threshold
    predictions_binary = np.stack([anomaly_scores > thresholds[i] for i in range(len(thresholds))], axis=0)

    # 2) Smoothing (optional)

    # 3) Calculate true anomaly sequences
    true_anomaly_sequences = calculate_anomaly_sequences(labels)

    # 4) Enlarge true anomaly sequences (optional)

    # 5) Calculate scores for each time steps based on NAB scoring function and true anomaly windows
    scores_time_steps = -1. * np.ones((len(labels)))
    for true_sequence in true_anomaly_sequences:
        start, end = true_sequence[0], true_sequence[1]
        len_seq = end - start
        for i in range(2*len_seq):
            scores_time_steps[min(start + i, len(labels) - 1)] = nab_scoring_function(3 * (i / len_seq) - 3)

    # 6) Sum all scores for every threshold
    scores_time_steps = np.stack([scores_time_steps for i in range(len(thresholds))], axis=0)
    # return scores_time_steps, predictions_binary
    scores_thresholds = scores_time_steps * predictions_binary
    for true_sequence in true_anomaly_sequences:
        start, end = true_sequence[0], true_sequence[1]
        detections = scores_thresholds[:, start:end+1] > 0
        first_detection_mask = detections.cumsum(axis=1).cumsum(axis=1) == 1
        scores_thresholds[:, start:end+1][~first_detection_mask] = 0
    scores_thresholds = np.sum(scores_thresholds, axis=1)

    # 7) Normalize by the number of true anomaly sequences
    if len(true_anomaly_sequences) != 0:
        scores_thresholds /= len(true_anomaly_sequences)

    # 8) Find maximum score
    idx = np.argmax(scores_thresholds)
    if return_threshold:
        return scores_thresholds[idx], thresholds[idx]
    else:
        return scores_thresholds[idx]

## Evaluation

In [36]:
results_total = {}
for dataset_id in dataset_ids:
    print(f"Now at dataset {dataset_id}")
    results = {}
    test_data = pd.read_csv(f'{data_folder_base_path}/{dataset_id}.test.csv')
    labels = test_data['is_anomaly'].to_numpy()
    if len(np.unique(labels)) == 1:
        continue
    approaches_paths = [x for x in os.listdir(f'{output_folder_path}/{dataset_id}') if not x.startswith('.') and not x in ['initial_weights', 'artificial_anomalies']]
    for approach_path in approaches_paths:
        model_id, learning_strategy_id, anomaly_score_id = approach_path.split('-')
        if anomaly_score_id == 'confidence_levels':
            continue
        if model_id not in results.keys():
            results[model_id] = {}
        if learning_strategy_id not in results[model_id].keys():
            results[model_id][learning_strategy_id] = {}
        if anomaly_score_id not in results[model_id][learning_strategy_id].keys():
            results[model_id][learning_strategy_id][anomaly_score_id] = {}
        run_date_id = [x for x in os.listdir(f'{output_folder_path}/{dataset_id}/{approach_path}') if not x.startswith('.')][0]
        score_path = f'{output_folder_path}/{dataset_id}/{approach_path}/{run_date_id}'
        anomaly_scores = pd.read_csv(f'{score_path}/anomaly_scores.csv').to_numpy()
        nonconformity_scores = pd.read_csv(f'{score_path}/nonconformity_scores.csv').to_numpy()
        start_idx = int(anomaly_scores[0, 0])
        number_of_elements = min(min(len(labels[start_idx:]), len(anomaly_scores)), len(nonconformity_scores))
        for scores in [("anomaly_scores", anomaly_scores), ("nonconformity_scores", nonconformity_scores)]:
            auc_score, pr_threshold, precision, recall = pr_auc(scores[1][:number_of_elements, 1], labels[start_idx:][:number_of_elements])
            nab_score, nab_threshold = nab_scoring(scores[1][:number_of_elements, 1], labels[start_idx:][:number_of_elements])
            results[model_id][learning_strategy_id][anomaly_score_id][scores[0]] = {
                "pr-auc": {
                    "threshold": pr_threshold,
                    "auc": auc_score,
                    "precision": precision,
                    "recall": recall,
                },
                "nab": {
                    "threshold": nab_threshold,
                    "score": nab_score,
                }
            }
    results_total[dataset_id] = results

Now at dataset machine-1-1
Now at dataset machine-1-2
Now at dataset machine-1-3
Now at dataset machine-1-4
Now at dataset machine-1-5


In [37]:
with open(f'../out/{dataset_category}/{collection_id}/{tuning_job_name}/results_total.json', 'w') as file:
    json.dump(results_total, file)

## Post Processing
Here we calculate results regarding
- ML model + learning strategy performance averaged across collection (for either anomaly likelihood or average)
- ML model performance averaged across collection + learning strategies (for either anomaly likelihood or average)
- LS performance averaged across collection + ML models (for either anomaly likelihood or average)
- Anomaly Likelihood vs Average

In [38]:
results_post_processing = {
    'models': {},
    'learning_strategies': {},
    'models_learning_strategies': {},
    'anomaly_scores': {},
    'nonconformity_scores': {}
}
template = {'auc': [], 'precision': [], 'recall': [], 'nab': []}
results_post_processing['nonconformity_scores']['default'] = deepcopy(template)
for dataset_id in results_total.keys():
    approaches_paths = [x for x in os.listdir(f'{output_folder_path}/{dataset_id}') if not x.startswith('.') and not x in ['initial_weights', 'artificial_anomalies']]
    for approach_path in approaches_paths:
        model_id, learning_strategy_id, anomaly_score_id = approach_path.split('-')
        model_ls_id = f'{model_id}-{learning_strategy_id}'
        if anomaly_score_id == 'confidence_levels':
            continue
        if model_id not in results_post_processing['models'].keys():
            results_post_processing['models'][model_id] = deepcopy(template)
        if learning_strategy_id not in results_post_processing['learning_strategies'].keys():
            results_post_processing['learning_strategies'][learning_strategy_id] = deepcopy(template)
        if model_ls_id not in results_post_processing['models_learning_strategies'].keys():
            results_post_processing['models_learning_strategies'][model_ls_id] = deepcopy(template)
        if anomaly_score_id not in results_post_processing['anomaly_scores'].keys():
            results_post_processing['anomaly_scores'][anomaly_score_id] = deepcopy(template)
        for dict_key, obj_id in [('models', model_id), ('learning_strategies', learning_strategy_id), ('models_learning_strategies', model_ls_id), ('anomaly_scores', anomaly_score_id)]:
            for pr_key in ['auc', 'precision', 'recall']:
                results_post_processing[dict_key][obj_id][pr_key].append(results_total[dataset_id][model_id][learning_strategy_id][anomaly_score_id]['anomaly_scores']['pr-auc'][pr_key])
                results_post_processing['nonconformity_scores']['default'][pr_key].append(results_total[dataset_id][model_id][learning_strategy_id][anomaly_score_id]['nonconformity_scores']['pr-auc'][pr_key])
            results_post_processing[dict_key][obj_id]['nab'].append(results_total[dataset_id][model_id][learning_strategy_id][anomaly_score_id]['anomaly_scores']['nab']['score'])
            results_post_processing['nonconformity_scores']['default']['nab'].append(results_total[dataset_id][model_id][learning_strategy_id][anomaly_score_id]['nonconformity_scores']['nab']['score'])
            
# Average across categories
for dict_key in results_post_processing.keys():
    for obj_key in results_post_processing[dict_key].keys():
        for score_key in template.keys():
            results_post_processing[dict_key][obj_key][score_key] = sum(results_post_processing[dict_key][obj_key][score_key]) / max(1, len(results_post_processing[dict_key][obj_key][score_key]))

In [39]:
with open(f'../out/{dataset_category}/{collection_id}/{tuning_job_name}/results_post_processing.json', 'w') as file:
    json.dump(results_post_processing, file)